<font color='blue'> **Step3_on multiple processors** </font> <font color='red'> **Docker issue** </font> en_gw3099_environment  

In [1]:
import sys ; import flopy ; import matplotlib.pyplot as plt ; import numpy as np ; from flopy.mf6.utils import Mf6Splitter
from pathlib import Path; import warnings  ; warnings.filterwarnings("ignore", category=DeprecationWarning)  # pip install h5py

In [2]:
ref_data4  = Path('E:/15_REPOS/00_BETAMI/bin/Flopy_3099/05_step02/parallel')  ; sim_ws = Path("./03c_step03") ; sim_ws.mkdir(exist_ok=True)  
ref_data5  = Path('E:/15_REPOS/00_BETAMI/bin/Flopy_3099/05_step02')

In [3]:
%%capture
parallel_sim = flopy.mf6.MFSimulation.load(sim_ws=ref_data4)   ; serial_sim   = flopy.mf6.MFSimulation.load(sim_ws=ref_data5)

**Set `path_to_mf6` to the path of parallel MODFLOW 6 (`path\to\mf6`)** </font>  <font color='red'> **otherwise `None`** </font>  

In [4]:
path_to_mf6 = 'none'
parallel_sim.run_simulation(processors = 6, silent=False)  # PETSc

FloPy is using the following executable to run the model: C:\Users\gross.DESKTOP-AIRQM2M\anaconda3\Scripts\mf6.exe
FloPy is using C:\Program Files\Microsoft MPI\Bin\mpiexec.exe to run C:\Users\gross.DESKTOP-AIRQM2M\anaconda3\Scripts\mf6.exe on 6 processors.






ERROR REPORT:
ERROR REPORT:
ERROR REPORT:



  1. Can not run parallel mode with this executable: no MPI
  1. Can not run parallel mode with this executable: no MPI
  1. Can not run parallel mode with this executable: no MPI


ERROR REPORT:

  1. Can not run parallel mode with this executable: no MPI


ERROR REPORT:

  1. Can not run parallel mode with this executable: no MPI


ERROR REPORT:

  1. Can not run parallel mode with this executable: no MPI


(False, [])

In [5]:
models = [parallel_sim.get_model(mname) for mname in parallel_sim.model_names]
times  = models[0].output.head().get_times()
heads  = [m.output.head().get_data(totim=times[-1]) for m in models]
hmin   = min([np.amin(h) for h in heads])
hmax   = max([np.amax(h[h < 1e30]) for h in heads])

AttributeError: 'NoneType' object has no attribute 'get_times'

## Reconstruct the data

The Model Splitter reconstruction functionality takes the following dictonary to merge the result back into one

In [ ]:
head_dict = dict(zip(range(0, len(heads)), heads))

Reconstruct the head data for the parallel run into a single array

In [ ]:
json_path = parallel_dir / "mfsplit_node_mapping.json"
mfsplit = Mf6Splitter(serial_sim)
mfsplit.load_node_mapping(parallel_sim, json_path)
reconstructed_head = mfsplit.reconstruct_array(head_dict)

Get the base reference results

In [ ]:
base_gwf = serial_sim.get_model()
base_head = base_gwf.output.head().get_data(totim=times[-1])

## Plot parallel versus serial simulated output

In [ ]:
fig = plt.figure(figsize=(figwidth, figheight * 1.3333))
dh = reconstructed_head - base_head
hv = [reconstructed_head, base_head, dh]
titles = ["Parallel", "Serial", "difference Parallel and Serial"]
for idx in range(3):
    ax = fig.add_subplot(3, 1, idx + 1)
    ax.set_aspect("equal")
    ax.set_title(titles[idx])

    if idx < 2:
        levels = np.arange(0, 100, 10)
        vmin = hmin
        vmax = hmax
    else:
        levels = None
        vmin = None
        vmax = None

    pmv = flopy.plot.PlotMapView(model=base_gwf, ax=ax, layer=0)
    h = pmv.plot_array(hv[idx], vmin=vmin, vmax=vmax)
    if levels is not None:
        c = pmv.contour_array(
            hv[idx],
            levels=np.arange(0, 100, 10),
            colors="white",
            linewidths=0.75,
            linestyles=":",
        )
        plt.clabel(c, fontsize=8)
    if base_gwf.modelgrid.idomain is not None:
        pmv.plot_inactive(color_noflow="white")
    plt.colorbar(h, ax=ax)
    if idx < 2:
        ax.set_xticklabels([])

plt.show(block=False)